# Obligatorio 2 - Big Data Science

Integrantes del grupo: Martín Brian - Joaquín Martínez

Debajo de cada pregunta o tarea incluya las celdas necesarias para desarrolar la respuesta. Puede usar una o varias celdas de código o mark down (https://www.datacamp.com/community/tutorials/markdown-in-jupyter-notebook)

Para entregar, renombrar este notebook como "Obligatorio 2 - Apellido1 - Apellido 2 - Apellido 3" con los apellidos de los miembros del grupo. Un solo integrante del grupo debe realizar la entrega. 


1. Cargar los datos del Obligatorio 1, de entrenamiento (.data) y validación (.test) en spark dataframes (distintos). Los nombres de las columnas deben corresponder a los especificados en "Attribute Information".

In [1]:
#from pyspark import SparkContext
import pyspark
from pyspark.sql import SQLContext

sc = pyspark.SparkContext.getOrCreate()

spark = SQLContext(sc)

In [2]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType

schema = StructType([
    StructField("age", IntegerType(), True),
    StructField("workclass", StringType(), True),
    StructField("fnlwgt", FloatType(), True),
    StructField("education", StringType(), True),
    StructField("education-num", FloatType(), True),
    StructField("marital-status", StringType(), True),
    StructField("occupation", StringType(), True),
    StructField("relationship", StringType(), True),
    StructField("race", StringType(), True),
    StructField("sex", StringType(), True),
    StructField("capital-gain", FloatType(), True),
    StructField("capital-loss", FloatType(), True),
    StructField("hours-per-week", FloatType(), True),
    StructField("native-country", StringType(), True),
    StructField("salary", StringType(), False)
])

In [3]:
#Leemos los archivos
df_data = spark.read.csv("./adult.data", header=False,schema=schema)
df_test = spark.read.csv("./adult.test", header=True,schema=schema)

In [4]:
df_data.toPandas()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516.0,Bachelors,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,2174.0,0.0,40.0,United-States,<=50K
1,50,Self-emp-not-inc,83311.0,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,United-States,<=50K
2,38,Private,215646.0,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50K
3,53,Private,234721.0,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,United-States,<=50K
4,28,Private,338409.0,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,40.0,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302.0,Assoc-acdm,12.0,Married-civ-spouse,Tech-support,Wife,White,Female,0.0,0.0,38.0,United-States,<=50K
32557,40,Private,154374.0,HS-grad,9.0,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0.0,0.0,40.0,United-States,>50K
32558,58,Private,151910.0,HS-grad,9.0,Widowed,Adm-clerical,Unmarried,White,Female,0.0,0.0,40.0,United-States,<=50K
32559,22,Private,201490.0,HS-grad,9.0,Never-married,Adm-clerical,Own-child,White,Male,0.0,0.0,20.0,United-States,<=50K


In [6]:
from pyspark.sql.functions import regexp_replace

# Limpiamos puntos en la columna salario
df_test = df_test.withColumn('salary', regexp_replace('salary', '<=50K.', '<=50K'))
df_test = df_test.withColumn('salary', regexp_replace('salary', '>50K.', '>50K'))

In [44]:
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml import Pipeline

vector_col = "corr_features"
fields_arr = [f for f in df_data.dtypes]
# for field in fields_arr:
#     print(field)
type_name = StringType.typeName()
categoric_fields = list(filter(lambda type: type[1]==type_name, fields_arr))
strings_to_index = [StringIndexer(inputCol=column[0], outputCol=column[0]+"_index") for column in categoric_fields]

assembler = VectorAssembler(inputCols=list(set(df_data.columns)-set(categoric_fields)), outputCol=vector_col)

print(f"{type_name}, {[field for field in fields_arr]}")
print(list(set(df_data.columns)-set(categoric_fields)))
print(categoric_fields)
print(strings_to_index)
print(assembler)
print(strings_to_index.extend([assembler]))

pipeline = Pipeline(stages=strings_to_index.extend([
    assembler
]))
print(pipeline.getStages())
pipeline.fit(df_data)

string, [('age', 'int'), ('workclass', 'string'), ('fnlwgt', 'float'), ('education', 'string'), ('education-num', 'float'), ('marital-status', 'string'), ('occupation', 'string'), ('relationship', 'string'), ('race', 'string'), ('sex', 'string'), ('capital-gain', 'float'), ('capital-loss', 'float'), ('hours-per-week', 'float'), ('native-country', 'string'), ('salary', 'string')]
['race', 'sex', 'capital-gain', 'education', 'hours-per-week', 'workclass', 'native-country', 'occupation', 'education-num', 'age', 'marital-status', 'capital-loss', 'relationship', 'fnlwgt', 'salary']
[('workclass', 'string'), ('education', 'string'), ('marital-status', 'string'), ('occupation', 'string'), ('relationship', 'string'), ('race', 'string'), ('sex', 'string'), ('native-country', 'string'), ('salary', 'string')]
[StringIndexer_a04f654dd74f, StringIndexer_1f2f7afba197, StringIndexer_0864639519d7, StringIndexer_d1b30118b08c, StringIndexer_3c9cb0bb7ac7, StringIndexer_67a1443ae2ad, StringIndexer_c02daa9

TypeError: 'NoneType' object is not iterable

2. Seleccionar un conjunto relevante de 5 atributos y crear un Spark Pipeline en el que el estimator sea un DecisionTreeClassifier (https://spark.apache.org/docs/latest/ml-classification-regression.html#decision-tree-classifier). Puede utilizar libremente los transformers/estimators de Spark para realizar ingeniería de atributos (StringIndexer, OneHotEncoding, etc).

3. Hacer el fit del pipeline con los datos de entrenamiento. 


In [5]:
#

4. Graficar la curva ROC utilizando los datos de validación (sin usar el paquete de evluación de Spark pyspark.ml.evaluation)

In [6]:
#

5. Escribir dos funciones train() y predict() que creen el arbol de acuerdo a la metodología vista en clase (utilizando entropía como métrica de homogeneidad de clases).
```
def train(train_dataframe):
    '''
    @return devuelve una estructura de datos que representa el árbol de decision
    '''
     pass        
```

```
def predict(tree, train_dataframe)
    '''
    @param tree la estructura de datos que representa el árbol de decisión.
    @ return un dataframe con todos los datos de train_dataframe con una columna adicional que representa la probabilidad de que el income sea >50K. 
    '''
    pass
```    
Ejemplo de uso:

```
tree = train(train_dataframe)
predictions_df = predict(tree, train_dataframe)
```

In [7]:
# 

<i>[Extra credit y alumnos masters]</i>
    
5.  Mejorar la implementación de su algoritmo evitando hacer el split cuando no se logra un mínimo de Information Gain ( https://en.wikipedia.org/wiki/Information_gain_in_decision_trees   )


In [8]:
#